In [4]:
from datetime import datetime

import pandas as pd
import requests

from api_key import api_key

### Stops

In [5]:
url = "https://api.um.warszawa.pl/api/action/dbstore_get"
params = {
    "id": "ab75c33d-3a26-4342-b36a-6e5fef0a3ac3",  # resource id given by the API
    "apikey": api_key,
}
stops = requests.get(url, params=params)
print(len(stops.json()["result"]))

8241


### Routes

In [6]:
url = "https://api.um.warszawa.pl//api/action/public_transport_routes"
params = {
    "apikey": api_key,
}
routes = requests.get(url, params=params)
print(len(routes.json()["result"]))

287


### Location

In [25]:
url = "https://api.um.warszawa.pl/api/action/busestrams_get"
params = {
    "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59",  # resource id given by the API
    "apikey": api_key,
    "type": 1,  # 1 - bus, 2 - tram
}
locations = requests.get(url, params=params)
print(len(locations.json()["result"]))

1893


### Timetable

In [8]:
url = "https://api.um.warszawa.pl//api/action/dbtimetable_get"
params = {
    "id": "e923fa0e-d96c-43f9-ae6e-60518c9f3238",  # resource id given by the API
    "apikey": api_key,
    "busstopId": "7074",
    "line": "159",
    "busstopNr": "02",
}
timetable = requests.get(url, params=params)
print(len(timetable.json()["result"]))

51


#### Buses

In [13]:
lines = [i["Lines"] for i in locations.json()["result"]]
lines = list(set(lines))
lines.sort()
print(lines)

['100', '102', '103', '104', '105', '106', '107', '108', '110', '111', '112', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '138', '139', '140', '141', '142', '143', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '197', '198', '199', '201', '202', '203', '204', '207', '208', '209', '210', '211', '212', '213', '217', '218', '219', '220', '221', '226', '228', '234', '239', '240', '245', '250', '251', '256', '262', '303', '308', '311', '317', '331', '338', '340', '401', '402', '409', '411', '414', '500', '502', '503', '504', '507', '509', '511', '512', '514', '516', '517', '518', '519', '520'

In [15]:
lines = [i["Lines"] for i in locations.json()["result"]]
lines = list(set(lines))
lines.sort()
print(lines)

['1', '10', '11', '13', '14', '15', '16', '17', '18', '2', '20', '22', '23', '24', '25', '26', '27', '28', '3', '31', '33', '36', '4', '6', '7', '79', '9']


In [29]:
df = pd.json_normalize(locations.json()["result"])

### NEEDS TO INSTALL fastparquet 

In [47]:
df.to_parquet(
    f"locations_{datetime.now().strftime('%Y%m%d%H%M%S')}.parquet",
    compression="zstd",
)